In [5]:
import $ivy.`com.typesafe.akka::akka-stream:2.6.4`
import $ivy.`com.typesafe.akka::akka-http:10.1.11`
// repl.pprinter() = repl.pprinter().copy(defaultHeight = 5 )

import $ivy.$                                     

import $ivy.$                                     
// repl.pprinter() = repl.pprinter().copy(defaultHeight = 5 )

In [6]:
import akka.stream._
import akka.stream.scaladsl._

import akka.stream._

import akka.stream.scaladsl._

In [7]:
import akka.{ Done, NotUsed }
import akka.actor.ActorSystem
import akka.util.ByteString
import scala.concurrent._
import scala.concurrent.duration._
import java.nio.file.Paths

import akka.{ Done, NotUsed }

import akka.actor.ActorSystem

import akka.util.ByteString

import scala.concurrent._

import scala.concurrent.duration._

import java.nio.file.Paths

In [8]:
import com.typesafe.config._

import com.typesafe.config._

In [13]:
implicit val system: ActorSystem = ActorSystem("QuickStart")

system: ActorSystem = akka://QuickStart

In [14]:
implicit val ec = system.dispatcher

ec: ExecutionContextExecutor = Dispatcher[akka.actor.default-dispatcher]

In [15]:
import akka.http.scaladsl.Http
import akka.http.scaladsl.model._
import scala.concurrent._
import scala.util._

import akka.http.scaladsl.Http

import akka.http.scaladsl.model._

import scala.concurrent._

import scala.util._

In [16]:

val QueueSize = 10

val poolClientFlow = 
    Http().cachedHostConnectionPool[Promise[HttpResponse]]("akka.io")

val queue =
  Source.queue[(HttpRequest, Promise[HttpResponse])](
          QueueSize, OverflowStrategy.dropNew)
    .via(poolClientFlow)
    .to(Sink.foreach({
      case ((Success(resp), p)) => p.success(resp)
      case ((Failure(e), p))    => p.failure(e)
    }))
    .run()


QueueSize: Int = 10
poolClientFlow: Flow[(HttpRequest, Promise[HttpResponse]), (Try[HttpResponse], Promise[HttpResponse]), Http.HostConnectionPool] = Flow(FlowShape(MapAsyncUnordered.in(1135870657),MapAsyncUnordered.out(431663193)))
queue: SourceQueueWithComplete[(HttpRequest, Promise[HttpResponse])] = akka.stream.impl.QueueSource$$anon$1@40dce5da

In [17]:
def queueRequest(request: HttpRequest): Future[HttpResponse] = {
  val responsePromise = Promise[HttpResponse]()
  queue.offer(request -> responsePromise).flatMap {
    case QueueOfferResult.Enqueued    => responsePromise.future
    case QueueOfferResult.Dropped     => Future.failed(new RuntimeException("Queue overflowed. Try again later."))
    case QueueOfferResult.Failure(ex) => Future.failed(ex)
    case QueueOfferResult.QueueClosed => Future.failed(new RuntimeException("Queue was closed (pool shut down) while running the request. Try again later."))
  }
}

defined function queueRequest

In [18]:
Source.queue[Int](5, OverflowStrategy.dropNew)

res17: Source[Int, SourceQueueWithComplete[Int]] = Source(SourceShape(queueSource.out(1793625467)))